In [ ]:
# 11/18/2021-1
from __future__ import absolute_import, division
from IPython.display import clear_output
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
from os.path import join
import tensorflow as tf
from tensorflow import keras

contentPath = os.getcwd()
dataPath = join(contentPath, 'data')

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [layers.RandomFlip("horizontal"), layers.RandomRotation(0.1),]
)

In [ ]:
base_model = keras.applications.MobileNetV2(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False,
)  # Do not include the ImageNet classifier at the top.

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)  # Apply random data augmentation

# Pre-trained Xception weights requires that input be scaled
# from (0, 255) to a range of (-1., +1.), the rescaling layer
# outputs: `(inputs * scale) + offset`
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(x)

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

model.summary()

In [ ]:
cv2Path=join(contentPath, 'CV2Images')
testPath=join(contentPath, 'images')

from tensorflow.keras.utils import image_dataset_from_directory
train_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()

validation_ds = image_dataset_from_directory(
    cv2Path,
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=456,
    image_size=(224, 224),
    batch_size=4)
print()

test_ds = image_dataset_from_directory(
    testPath,
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=1)
print()
# clear_output()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 3
model.fit(train_ds, epochs=epochs, validation_data=validation_ds)
# clear_output()

In [ ]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
# model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[keras.metrics.BinaryAccuracy()],
)

epochs = 1
history=model.fit(train_ds,
                  epochs=epochs,
                  validation_data=validation_ds)

In [ ]:
from CV2_Utils import *
import cv2
testPath= join(contentPath, 'friendly_validation_13.png')
img=cv2.imread(testPath)
img=cv2.resize(img, dsize=(224, 224))
# cvu.plotShowSingleImage(img, title1=img.shape)
# result=model.predict(x=img, shape=(None,224,224,3))
sleep(1)
img=None

In [ ]:
modelName = base_model.name + '_retrained.h5'
savePath = join(contentPath, modelName)
print(savePath)
# model.save(savePath)

In [ ]:
# serialize model to JSON
model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
model.save_weights("model.h5")

print("Saved model to disk")

In [ ]:
from tensorflow.keras.models import load_model

model.save('model.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
newModel = load_model('model.h5')

In [ ]:

newModel.predict(test_ds.take(1))

In [ ]:
# print(batchSize)
batchSize=len(image_batch)
# Retrieve a batch of images from the test set
image_batch, label_batch = test_ds.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(batchSize):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

In [ ]:
import numpy as np
from keras.preprocessing import image
img= cv2.imread('test_0071.jpg')
cvu.plotShowSingleImage(img)
img_width, img_height = 224, 224
img = image.load_img('test_0071.jpg', target_size = (img_width, img_height))
print(img)
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
print(img.shape)

result = model.predict(img)
print(result)